# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
!pip install ipywidgets
!pip install transformers
!pip install sentencepiece
!pip install sacremoses
!pip install ipywidgets transformers sentencepiece
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline
!pip install transformers sentencepiece torch datasets

  Using cached ipywidgets-8.1.2-py3-none-any.whl (139 kB)
  Using cached widgetsnbextension-4.0.10-py3-none-any.whl (2.3 MB)
  Using cached jupyterlab_widgets-3.0.10-py3-none-any.whl (215 kB)
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/opt/venv/lib/python3.10/site-packages/widgetsnbextension'
Check the permissions.


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/opt/venv/lib/python3.10/site-packages/sacremoses'
Check the permissions.


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip in

ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/opt/venv/lib/python3.10/site-packages/widgetsnbextension'
Check the permissions.


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  Using cached datasets-2.17.1-py3-none-any.whl (536 kB)
  Using cached huggingface_hub-0.20.3-py3-none-any.whl (330 kB)
  Using cached pyarrow-15.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (38.3 MB)
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
  Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
  Using cached aiohttp-3.9.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached frozenlist-1.4.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (239 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached yarl-1.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (301 kB)
  Using cached multidict-6.0.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (124 kB)
  Using cached async_timeout-4.0.3-py3-none

### Get data from `.csv` files and then preprocess data

In [2]:
def preprocess_data(file_path, name1, name2, name3):
    
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    
    #file path
    dir1 = f"{file_path}{name1}.csv"
    dir2 = f"{file_path}{name2}.csv"
    dir3 = f"{file_path}{name3}.csv"
    
    # Read CSV files
    movie_reviews_name1 = pd.read_csv(dir1)
    movie_reviews_name2 = pd.read_csv(dir2)
    movie_reviews_name3 = pd.read_csv(dir3)
    
    # get columns names
    list1=movie_reviews_name1.columns
    list2=movie_reviews_name2.columns
    list3=movie_reviews_name3.columns

    # rename the columns names to be the same as acquired 
    movie_reviews_name1= movie_reviews_name1.rename(columns={list1[0]:'Title',
                                                             list1[1]:'Year',
                                                             list1[2]:'Synopsis',
                                                             list1[3]:'Review'})

    movie_reviews_name2 = movie_reviews_name2.rename(columns={list2[0]: 'Title',
                                                                list2[1]: 'Year',
                                                                list2[2]: 'Synopsis',
                                                                list2[3]: 'Review'})
    movie_reviews_name3=movie_reviews_name3.rename(columns={list3[0]: 'Title',
                                                            list3[1]: 'Year',
                                                            list3[2]: 'Synopsis',
                                                            list3[3]: 'Review'})

    # Add a column to each dataframe indicating the original language
    movie_reviews_name1["Original Language"] = f"{name1}"
    movie_reviews_name2["Original Language"] = f"{name2}"
    movie_reviews_name3["Original Language"] = f"{name3}"

    # Combine dataframes
    movie_reviews = pd.concat([movie_reviews_name1, movie_reviews_name2, movie_reviews_name3], ignore_index=True)
    return movie_reviews,movie_reviews_name1,movie_reviews_name2,movie_reviews_name3


In [3]:
file_path = 'data/movie_reviews_'

df = preprocess_data(file_path, "eng", "fr", "sp")[0]
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",eng
23,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin...",sp
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",eng
25,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",sp
21,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",sp
16,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",fr
15,Le Dîner de Cons,1998,Le film suit l'histoire d'un groupe d'amis ric...,"""Je n'ai pas aimé ce film du tout. Le concept ...",fr
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",fr
18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",fr
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",eng


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [7]:
#Create a function to translate text
def translate(text, model, tokenizer, target_language="en"):
    translated = model.generate(**tokenizer.prepare_seq2seq_batch([text], return_tensors="pt"))
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Initialize models and tokenizers for French and Spanish translation
tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
model_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
tokenizer_sp = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
model_sp = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-es-en")


/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [12]:
# Filter reviews in French and translate to English
fr_reviews = df[df["Original Language"] == "fr"]["Review"]
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, model_fr, tokenizer_fr))

# Filter synopsis in French and translate to English
fr_synopsis = df[df["Original Language"] == "fr"]["Synopsis"]
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, model_fr, tokenizer_fr))

# Filter reviews in Spanish and translate to English
es_reviews = df[df["Original Language"] == "sp"]["Review"]
es_reviews_en = es_reviews.apply(lambda x: translate(x, model_sp, tokenizer_sp))

# Filter synopsis in Spanish and translate to English
es_synopsis = df[df["Original Language"] == "sp"]["Synopsis"]
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, model_sp, tokenizer_sp))


In [ ]:
# Translate French and Spanish reviews and synopses
for index, row in df.iterrows():
    if row["Original Language"] == "fr":
        df.at[index, "Synopsis"] = translate(row["Synopsis"], model_fr, tokenizer_fr)
        df.at[index, "Review"] = translate(row["Review"], model_fr, tokenizer_fr)
    elif row["Original Language"] == "sp":
        df.at[index, "Synopsis"] = translate(row["Synopsis"], model_sp, tokenizer_sp)
        df.at[index, "Review"] = translate(row["Review"], model_sp, tokenizer_sp)

In [13]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",sp
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp
15,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ...",fr
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",fr
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",fr
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",eng
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",eng
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",fr
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",eng
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [20]:

sentiment_analysis = pipeline("sentiment-analysis")

# # Define a function to perform sentiment analysis on text
def analyze_sentiment(text, classifier):
#     """
#     Function to perform sentiment analysis on a text 
#     """

# # Perform sentiment analysis on reviews and store results in a new column
    df["Sentiment"] = df["Review"].apply(lambda x: sentiment_analysis(x)[0]["label"])

    return df

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [22]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",fr,NEGATIVE
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",eng,NEGATIVE
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",sp,NEGATIVE
15,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ...",fr,NEGATIVE
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",fr,POSITIVE
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",fr,NEGATIVE
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp,NEGATIVE
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",eng,NEGATIVE
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",fr,NEGATIVE
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",eng,POSITIVE


In [23]:
#export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)
